In [1]:
import numpy as np
import os
import cv2
import glob
import tensorflow as tf
from ultralytics import YOLO

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.models import load_model


In [2]:
p=os.getcwd()+"/data"
#p=os.getcwd()+"/prep"
seed=20
batch_size = 32
image_height = 224
image_width = 224
dir_names = sorted(os.listdir(p))
dir_names

['crab', 'dog', 'dolphin', 'elephant', 'human', 'lion', 'snake']

In [17]:

import splitfolders

input_folder = "data" 
output_folder = "animals_dataset"

splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.8, .2), group_prefix=None)


class_names = dir_names
class_num=len(class_names)
class_names,class_num

(['crab', 'dog', 'dolphin', 'elephant', 'human', 'lion', 'snake'], 7)

In [ ]:

def train_model():
    model = YOLO('yolov8n-cls.pt') 

    results = model.train(
        data="animals_dataset",
        epochs=20,
        imgsz=224,
        batch=16,
        name='my_animal_model'
    )

train_model()

x:\Anaconda\envs\tens_env\lib\site-packages\keras\src\layers\preprocessing\data_layer.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Ultralytics 8.4.5  Python-3.10.19 torch-2.5.1 CPU (Intel Core i5-10300H 2.50GHz)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=animals_dataset, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=my_animal_model2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True

In [12]:
import time

MODEL_PATH = 'runs/classify/my_animal_model2/weights/best.pt' 


print("Загрузка модели...")
model = YOLO(MODEL_PATH)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

if not cap.isOpened():
    print("Error: Can't receive frame.")
    exit()

font = cv2.FONT_HERSHEY_SIMPLEX

while True:
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    display_frame = frame.copy()

    results = model.predict(display_frame, verbose=False, stream=False)

    result = results[0]

    
    top5_indices = result.probs.top5
    top5_conf = result.probs.top5conf
    

    top1_idx = top5_indices[0]
    top1_conf = top5_conf[0].item()
    top1_name = result.names[top1_idx]


    if len(top5_indices) > 1:
        top2_idx = top5_indices[1]
        top2_conf = top5_conf[1].item()
        top2_name = result.names[top2_idx]
    else:
        top2_name = "N/A"
        top2_conf = 0.0

    cv2.rectangle(display_frame, (0, 0), (400, 100), (0, 0, 0), -1)
    
    text_1 = f"1. {top1_name}: {top1_conf * 100:.1f}%"
    cv2.putText(display_frame, text_1, (10, 40), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

    text_2 = f"2. {top2_name}: {top2_conf * 100:.1f}%"
    cv2.putText(display_frame, text_2, (10, 80), font, 0.8, (0, 255, 255), 2, cv2.LINE_AA)

    fps = 1.0 / (time.time() - start_time)
    cv2.putText(display_frame, f"FPS: {fps:.1f}", (10, 130), font, 0.6, (200, 200, 200), 1)

    cv2.imshow("Animal Classifier", display_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Загрузка модели...


In [10]:
import cv2
import numpy as np
from ultralytics import YOLO
import time

CLASSIFIER_PATH = 'runs/classify/my_animal_model2/weights/best.pt'

DETECTOR_PATH = 'yolov8n.pt' 



print("Загрузка моделей...")
detector = YOLO(DETECTOR_PATH)
classifier = YOLO(CLASSIFIER_PATH)

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

if not cap.isOpened():
    print("Error: Can't receive frame.")
    exit()

font = cv2.FONT_HERSHEY_SIMPLEX

while True:
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    display_frame = frame.copy()
    
    det_results = detector.predict(frame, verbose=False, conf=0.4)
    
    best_box = None
    max_area = 0

    if len(det_results[0].boxes) > 0:
        for box in det_results[0].boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            
            area = (x2 - x1) * (y2 - y1)
            if area > max_area:
                max_area = area
                best_box = (x1, y1, x2, y2)

    if best_box:
        x1, y1, x2, y2 = best_box
        
        h, w, _ = frame.shape
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)

        object_crop = frame[y1:y2, x1:x2]

        if object_crop.size > 0:
            cls_results = classifier.predict(object_crop, verbose=False)
            
            probs = cls_results[0].probs
            top1_idx = probs.top1
            conf = probs.top1conf.item()
            class_name = cls_results[0].names[top1_idx]


            cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

            label = f"{class_name}: {conf*100:.1f}%"
            
            (tw, th), _ = cv2.getTextSize(label, font, 0.8, 2)
            cv2.rectangle(display_frame, (x1, y1 - 30), (x1 + tw, y1), (0, 255, 0), -1)
            cv2.putText(display_frame, label, (x1, y1 - 5), font, 0.8, (0, 0, 0), 2)
            
            try:
                debug_crop = cv2.resize(object_crop, (150, 150))
                display_frame[10:160, 10:160] = debug_crop
                cv2.rectangle(display_frame, (10, 10), (160, 160), (255, 255, 255), 1)
            except:
                pass

    else:

        cv2.putText(display_frame, "Searching for animal...", (20, 50), font, 1, (0, 0, 255), 2)

    fps = 1.0 / (time.time() - start_time)
    cv2.putText(display_frame, f"FPS: {fps:.1f}", (display_frame.shape[1] - 150, 40), font, 0.7, (200, 200, 200), 2)

    cv2.imshow("Smart Animal Classifier", display_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Загрузка моделей...
